<a href="https://colab.research.google.com/github/CapArrow/HighCountingVQA/blob/main/create_dataset_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
import requests
import os
import zipfile
import json
import numpy as np
from tqdm import tqdm
import pandas as pd
from imutils import paths

In [53]:
# mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
# change directory to the Colab Notebooks folder with link to drive of lars
%cd /content/drive/My Drive/MMDL2024/Colab Notebooks/

/content/drive/.shortcut-targets-by-id/1LQRkg9gduygZjO84k_knGQGpy1qEPTp6/Colab Notebooks


In [48]:
# load json
train_file = open("tallyqa_train.json")
test_file = open("tallyqa_test.json")
train_data = np.array(json.load(train_file))
test_data = np.array(json.load(test_file))
train_file.close()
test_file.close()

# test file has one additional key: "issimple"
# in the following this key is ignored
print(train_data[0].keys())
print(test_data[0].keys())


# concatenate data
data_dict = np.concatenate((train_data, test_data), axis=0)
# get dict keys
data_keys = data_dict[0].keys()
# transform dicts to list and get data array
df = pd.json_normalize(data_dict)
data = df.to_numpy()[:, :6]

dict_keys(['image', 'answer', 'data_source', 'question', 'image_id', 'question_id'])
dict_keys(['image', 'answer', 'data_source', 'question', 'image_id', 'question_id', 'issimple'])


In [49]:
# test if all image files are available
data_image_names = np.unique(data[:, 0])
np.random.shuffle(data_image_names)
print(len(data), len(data_image_names))

filenotfound = []

for name in tqdm(data_image_names):
  folder, image_name = name.split("/")
  if folder == "VG_100K" or folder == "VG_100K_2":
    name = "visual_genome/" + name
  else:
    name = "coco/" + name
  if os.path.isfile(name):
    filenotfound.append(name)

287907 159432


100%|██████████| 159432/159432 [00:29<00:00, 5446.86it/s]


In [50]:
print(str(len(filenotfound)) + " files not found!")
print(np.array(filenotfound)[:10])

139117 files not found!
['coco/train2014/COCO_train2014_000000401969.jpg'
 'coco/val2014/COCO_val2014_000000562879.jpg'
 'visual_genome/VG_100K/2316650.jpg' 'visual_genome/VG_100K/2358729.jpg'
 'visual_genome/VG_100K_2/2390799.jpg'
 'visual_genome/VG_100K_2/2413673.jpg'
 'coco/train2014/COCO_train2014_000000495903.jpg'
 'visual_genome/VG_100K/2343969.jpg'
 'coco/train2014/COCO_train2014_000000160573.jpg'
 'coco/train2014/COCO_train2014_000000274801.jpg']


In [51]:
# extract every question with answer greater then 4
gcd = data[:, 1] >= 4
greater_count_data = data[gcd]
lesser_count_data = data[np.logical_not(gcd)]
np.random.shuffle(lesser_count_data)
# create new data array with all answers greater then 4 and 20.000 answers lower then 4
new_data = np.concatenate((greater_count_data, lesser_count_data[:20000]), axis=0)
print(new_data.shape)

(61528, 6)


In [52]:
# create new json file for new_data
file = open("HighCountVQA.json", "w")
json_data = []

for line in tqdm(new_data):
  tmp_dict = dict()
  for i, key in enumerate(data_keys):
    tmp_dict[key] = line[i]
  json_data.append(tmp_dict)

json.dump(json_data, file)

100%|██████████| 61528/61528 [00:00<00:00, 238508.93it/s]
